In [1]:
import numpy as np 
import tensorflow as tf
from spektral.layers import GCNConv
from sklearn.metrics import f1_score
from keras import backend as K

In [32]:
from tensorflow.keras.layers import Layer
import tensorflow as tf

class CustomGrapheConv(Layer):
    def __init__(self, output_dim, canaux, **kwargs):
        self.output_dim = output_dim
        self.cannaux = canaux
        super(CustomGrapheConv, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        assert len(input_shape) == 2          # we want to inputs
        input_shape_1, input_shape_2 = input_shape
        D = input_shape_1[2]
        K = self.output_dim
        canaux = self.cannaux
        print('D:', D)
        print('K:', K)
        print('cannaux:', canaux)

        # Créer les poids de la couche (variables)
        self.U = self.add_weight(name='U',
                                        shape=(D, K),
                                        initializer='uniform',
                                        trainable=True)
        
        self.Cs = []

        for k in range(canaux):
            C = self.add_weight(name='C_'+str(k),
                                        shape=(D, K),
                                        initializer='uniform',
                                        trainable=True)
            self.Cs.append(C)

        super(CustomGrapheConv, self).build(input_shape)

    def call(self, inputs):
        input_1, input_2 = inputs
        output_1 = tf.matmul(input_1, self.U)
        #output_2 = tf.matmul(input_2, self.kernel_2)

        # Combinaison des sorties
        #output = output_1 + output_2

        return output_1

    def compute_output_shape(self, input_shape):
        # Définir la forme de sortie de la couche
        return (input_shape[0][0], self.output_dim)

# Exemple d'utilisation de la couche personnalisée dans un modèle
input_1 = tf.keras.Input(shape=(None,10))
input_2 = tf.keras.Input(shape=(None, None,1))

couche_personnalisee = CustomGrapheConv(output_dim=5,canaux=16)([input_1, input_2])

# Création du modèle
model = tf.keras.Model(inputs=[input_1, input_2], outputs=couche_personnalisee)

# Compiler le modèle
model.compile(optimizer='adam', loss='mse')

# Afficher la description du modèle
model.summary()


D: 10
K: 5
cannaux: 16
Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_59 (InputLayer)       [(None, None, 10)]           0         []                            
                                                                                                  
 input_60 (InputLayer)       [(None, None, None, 1)]      0         []                            


                                                                                                  
 custom_graphe_conv_2 (Cust  (None, None, 5)              850       ['input_59[0][0]',            
 omGrapheConv)                                                       'input_60[0][0]']            
                                                                                                  
Total params: 850 (3.32 KB)
Trainable params: 850 (3.32 KB)
Non-trainable params: 0 (0.00 Byte)
__________________________________________________________________________________________________
